# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests


import warnings
warnings.filterwarnings("ignore")

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,waitangi,-43.9535,-176.5597,53.28,71,100,8.75,NZ,1715240843
1,1,piacabucu,-10.4056,-36.4344,77.34,92,57,8.95,BR,1715240846
2,2,longyearbyen,78.2186,15.6401,33.64,69,40,21.85,SJ,1715240848
3,3,olonkinbyen,70.9221,-8.7187,26.51,89,100,32.79,SJ,1715240849
4,4,itoman,26.1247,127.6694,77.77,54,75,18.41,JP,1715240851


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# Configure the map plot
vacation_humidity_map = city_data_df.hvplot.points('Lng',
                                                   'Lat', 
                                                   geo = True,
                                                   frame_width = 800,
                                                   frame_height = 600,
                                                   size = 'Humidity',
                                                   tiles = 'OSM',
                                                   scale = 1,
                                                   color = 'City',
                                                   alpha = 0.50
                                                  )

# Display the map
vacation_humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
column_titles = city_data_df.columns.tolist()
column_titles

['City_ID',
 'City',
 'Lat',
 'Lng',
 'Max Temp',
 'Humidity',
 'Cloudiness',
 'Wind Speed',
 'Country',
 'Date']

In [5]:
ideal_weather_df = city_data_df.loc[(city_data_df["Max Temp"] > 70) & (city_data_df["Max Temp"] < 80) & (city_data_df["Cloudiness"] == 0), :]
# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
print(len(ideal_weather_df))
ideal_weather_df.head()

9


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
229,229,ha'il,27.5219,41.6907,78.82,22,0,13.80,SA,1715241242
271,271,regente feijo,-22.2214,-51.3028,74.48,57,0,8.05,BR,1715241257
272,272,arar,30.9753,41.0381,78.82,12,0,11.50,SA,1715241317
285,285,timimoun,29.2639,0.2310,71.98,18,0,17.27,DZ,1715241338
302,302,laguna,38.4210,-121.4238,70.09,28,0,14.97,US,1715241369


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
229,ha'il,SA,27.5219,41.6907,22,
271,regente feijo,BR,-22.2214,-51.3028,57,
272,arar,SA,30.9753,41.0381,12,
285,timimoun,DZ,29.2639,0.2310,18,
302,laguna,US,38.4210,-121.4238,28,
344,letlhakeng,BW,-24.0944,25.0298,23,
443,linxia chengguanzhen,CN,35.6003,103.2064,19,
514,fort stockton,US,30.8940,-102.8793,28,
544,warmbaths,ZA,-24.8833,28.2833,25,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "api_key": geoapify_key,
    "categories": "accomodation.hotel",
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    hotel_latitude = row["Lat"]
    hotel_longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{hotel_longitude},{hotel_latitude},{radius}"
    params["bias"] = f"proximity:{hotel_longitude},{hotel_latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ha'il - nearest hotel: No hotel found
regente feijo - nearest hotel: No hotel found
arar - nearest hotel: No hotel found
timimoun - nearest hotel: No hotel found
laguna - nearest hotel: No hotel found
letlhakeng - nearest hotel: No hotel found
linxia chengguanzhen - nearest hotel: No hotel found
fort stockton - nearest hotel: No hotel found
warmbaths - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
229,ha'il,SA,27.5219,41.6907,22,No hotel found
271,regente feijo,BR,-22.2214,-51.3028,57,No hotel found
272,arar,SA,30.9753,41.0381,12,No hotel found
285,timimoun,DZ,29.2639,0.2310,18,No hotel found
302,laguna,US,38.4210,-121.4238,28,No hotel found
344,letlhakeng,BW,-24.0944,25.0298,23,No hotel found
443,linxia chengguanzhen,CN,35.6003,103.2064,19,No hotel found
514,fort stockton,US,30.8940,-102.8793,28,No hotel found
544,warmbaths,ZA,-24.8833,28.2833,25,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
# Configure the map plot
hotel_map = hotel_df.hvplot.points('Lng',
                                   'Lat', 
                                   geo = True,
                                   frame_width = 800,
                                   frame_height = 600,
                                   size = 'Humidity',
                                   tiles = 'OSM',
                                   scale = 1,
                                   color = 'City',
                                   alpha = 0.50,
                                   hover_cols = ['City', 'Hotel Name', 'Country']
                                )

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)